In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping

movie_data = pd.read_csv('/kaggle/input/mooooovies/MovieCleanData.csv')

#Train Word2Vec model on movie titles and descriptions
def train_word2vec(data):
    sentences = [title.split() for title in data['title']] + [desc.split() for desc in data['description']]
    return Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

word2vec_model = train_word2vec(movie_data)


#Neural collaborative filtering
def NCF_model(num_movies, embedding_size):
    movie_input = Input(shape=(1,))
    movie_embedding = Embedding(num_movies, embedding_size)(movie_input)
    movie_vec = Flatten()(movie_embedding)
    dense = Dense(128, activation='relu')(movie_vec)
    dense = Dense(64, activation='relu')(dense) 
    output = Dense(1)(dense)
    model = Model(movie_input, output)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

num_movies = movie_data['dataId'].nunique()
ncf_model = NCF_model(num_movies, 50)
ncf_model.fit(movie_data['dataId'].values, movie_data['rating'].values, epochs=32, batch_size=64, verbose=1)


#Function to get similar movies based on description using Word2Vec
def get_similar_movies(model, movie_title, data):
    movie_vec = model.wv[movie_title.split()]
    movie_vec_mean = np.mean(movie_vec, axis=0)
    similarities = []
    indices = []
    for idx, row in data.iterrows():
        desc_words = row['description'].split()
        if desc_words:
            desc_vec = model.wv[desc_words]
            desc_vec_mean = np.mean(desc_vec, axis=0)
            sim_score = cosine_similarity([movie_vec_mean], [desc_vec_mean])[0][0]
            similarities.append(sim_score)
            indices.append(idx)
    return indices, similarities


#Recommendation function using similarity
def recommend_movies(genre, min_rating, similar_movie):
    filtered_data = movie_data[(movie_data['genre'].str.contains(genre, case=False, na=False)) & (movie_data['rating'] >= min_rating)]
    filtered_data = filtered_data.reset_index(drop=True)

    if similar_movie and similar_movie in word2vec_model.wv.key_to_index:
        indices, similarities = get_similar_movies(word2vec_model, similar_movie, filtered_data)
        if indices:
            similar_data = filtered_data.loc[indices]
            similar_data['similarity'] = similarities
            similar_data = similar_data.sort_values(by=['similarity', 'rating'], ascending=[False, False])
            return similar_data.head(5)
        else:
            return pd.DataFrame()
    return filtered_data.nlargest(5, 'rating')

#Function to ask the user for input
def ask_user():
    print("Please answer the following questions for movie recommendations:")
    preferred_genre = input("1. What genre do you prefer? ")
    minimum_rating = float(input("2. What is the minimum rating you would prefer? "))
    similar_movie = input("3. Do you have a similar movie in mind? (If not, leave blank) ")
    recommendations = recommend_movies(preferred_genre, minimum_rating, similar_movie)
    counter = 0
    
    if not recommendations.empty:
        print("\nRecommended Movies:\n")
        for index, movie in recommendations.iterrows():
            print(f"Movie {counter + 1}:")
            print(f"Title: {movie['title']}")
            print(f"Genre: {movie['genre']}")
            movie_length = movie.get('length', 'Not Available')
            movie_length = "Not Available" if movie_length == -1 else movie_length
            print(f"Movie Length: {movie_length} minutes")  
            print(f"Release Year: {movie['releaseYear']}")
            print(f"Rating: {movie['rating']}")
            print(f"Description: {movie['description']}\n")
            print('-' * 50)
            counter += 1
    else:
        print("\nNo recommendations found based on the criteria.")


Epoch 1/32
824/824 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - loss: 8.4429
Epoch 2/32
824/824 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - loss: 1.1329
Epoch 3/32
824/824 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 0.9131
Epoch 4/32
824/824 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - loss: 0.7885
Epoch 5/32
824/824 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - loss: 0.7587
Epoch 6/32
824/824 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - loss: 0.7451
Epoch 7/32
824/824 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 0.7425
Epoch 8/32
824/824 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - loss: 0.7423
Epoch 9/32
824/824 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - loss: 0.7312
Epoch 10/32
824/824 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 0.7330
Epoch 11/32
824/824 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 0.7141
Epoch 12/32
824/824 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - loss: 0.7068
Epoch 13/32
824/824 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - loss: 0.7178
Epoch 14/32
824/824 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - loss: 0.7157
Epoch 15/32
824/824 ━━━━━━━━━━━━━

In [14]:
ask_user()

Please answer the following questions for movie recommendations:


1. What genre do you prefer?  comedy
2. What is the minimum rating you would prefer?  7
3. Do you have a similar movie in mind? (If not, leave blank)  friends



Recommended Movies:

Movie 1:
Title: Wayne's World
Genre: Comedy
Movie Length: 94 minutes
Release Year: 1992
Rating: 7.0
Description: Two slacker friends try to promote their public-access cable show.

--------------------------------------------------
Movie 2:
Title: Workin' Moms
Genre: Comedy
Movie Length: 30 minutes
Release Year: 2017
Rating: 7.8
Description: Four very different thirtysomething working-mother friends try to balance their jobs, family lives, and love lives in modern-day Toronto, Canada.

--------------------------------------------------
Movie 3:
Title: College Romance
Genre: Comedy
Movie Length: 30 minutes
Release Year: 2018
Rating: 8.4
Description: Three best friends look for love, laughs and some lifelong memories while attending college together.

--------------------------------------------------
Movie 4:
Title: Upstairs
Genre: Comedy
Movie Length: 22 minutes
Release Year: 2013
Rating: 7.2
Description: Four college students spending their life together.

------

In [ ]:
'''

Student names   |   Student ID
--------------  | --------------
Khalid Nimri    |   2140145
Aseel Suhail    |   2140197

'''